# Data Augmentation

In [1]:
import os
import cv2
import glob
import keras
from keras.utils.image_utils import img_to_array, array_to_img, load_img
from keras.preprocessing.image import ImageDataGenerator

In [2]:
#STARE
def data_augmentation():
    img_dir = '../data/filtered-data/ODIR/Pure AMD' #orginal directory
    aug_dir = '../data/filtered-data/ODIR/Pure AMD/Augmented' #augmented directory
    data_path = os.path.join(img_dir,'*g')

    files = glob.glob(data_path) #error @ rgb_img when removing glob.glob dunno y
    data = []
    for f1 in files:
        img = cv2.imread(f1) #read img
        rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) #convert BGR img to RGB (depends)
        data.append(rgb_img) 

        x = img_to_array(rgb_img)  
        x = x.reshape((1,) + x.shape)  

        path, dirs, files = next(os.walk(img_dir))
        file_count = len(files)

        datagen = ImageDataGenerator(horizontal_flip = True)

        i = 0
        for batch in datagen.flow(x, batch_size = 1, save_to_dir = aug_dir, save_prefix = 'ODIR_Augment', save_format = 'jpg'):
            i += 1
            if i == 1:
                break  

# Red Channel Extraction & CLAHE

In [2]:
import numpy as np
import os
from os import listdir

In [3]:
train_data = []

In [4]:
pure_amd = []
normal = []
amd_w_others = []

In [5]:
def rce_clahe(img_dir, red_dir, data_family, data_class):
    if not os.path.exists(red_dir):
        os.mkdir(red_dir)

    datagen = ImageDataGenerator(horizontal_flip = True)

    i = 0
    for images in os.listdir(img_dir): 
        # check if the image ends with png or jpg or jpeg
        if (images.endswith(".png") or images.endswith(".jpg")\
            or images.endswith(".jpeg")):
            img = cv2.imread(img_dir + "/" + images, cv2.IMREAD_UNCHANGED)
            resized_img = cv2.resize(img, (512, 512))

            # exclude red channel
            b,g,r = cv2.split(resized_img)

            # apply CLAHE
            clahe = cv2.createCLAHE(clipLimit=5)
            b_clahe_enhanced = clahe.apply(b) + 30 # increase intensity of contrast
            g_clahe_enhanced = clahe.apply(g) + 30
            r_clahe_enhanced = clahe.apply(r)

            # merge enhanced channels
            clahe_enhanced_img = cv2.merge((b_clahe_enhanced, g, r_clahe_enhanced))

            grayscale = cv2.cvtColor(clahe_enhanced_img, cv2.COLOR_BGR2GRAY)

            if data_class == 'pure_amd':
                pure_amd.append(grayscale)
            elif data_class == 'normal':
                normal.append(grayscale)
            elif data_class == 'amd_w_others':
                amd_w_others.append(grayscale)

            path = os.path.join(red_dir, "{}_CLAHE_".format(data_family) + images.split('.')[0] + ".png")
            cv2.imwrite(path , grayscale)
            i+=1

In [8]:
#rce_clahe('../data/preprocessing/filtered-data/ODIR/Pure AMD', '../data/preprocessing/CLAHE/ODIR/Pure AMD', 'ODIR', 'pure_amd')
#rce_clahe('../data/preprocessing/filtered-data/ODIR/AMD w others', '../data/preprocessing/CLAHE/ODIR/AMD w others', 'ODIR', 'amd_w_others')
#rce_clahe('../data/preprocessing/filtered-data/ODIR/Normal', '../data/preprocessing/CLAHE/ODIR/Normal', 'ODIR', 'normal')

#rce_clahe('../data/preprocessing/filtered-data/RFMiD/Pure AMD', '../data/CLAHE/RFMiD/Pure AMD', 'RFMiD', 'pure_amd')
#rce_clahe('../data/preprocessing/filtered-data/RFMiD/AMD w others', '../data/preprocessing/CLAHE/RFMiD/AMD w others', 'RFMiD', 'amd_w_others')
#rce_clahe('../data/preprocessing/filtered-data/RFMiD/Normal', '../data/preprocessing/CLAHE/RFMiD/Normal', 'RFMiD', 'normal')
#
rce_clahe('../data/preprocessing/filtered-data/STARE/AMD w others', '../data/preprocessing/CLAHE/STARE/AMD w others', 'QSTARE', 'amd_w_others')
#rce_clahe('../data/preprocessing/filtered-data/STARE/Normal', '../data/preprocessing/CLAHE/STARE/Normal', 'STARE', 'normal')

In [10]:
def training_data(img_dir, data_class):
    for images in os.listdir(img_dir): 
        # check if the image ends with png or jpg or jpeg
        if (images.endswith(".png") or images.endswith(".jpg")\
            or images.endswith(".jpeg")):
            img = cv2.imread(img_dir + "/" + images, cv2.IMREAD_UNCHANGED)
            resized_img = cv2.resize(img, (512, 512))

            # exclude red channel
            b,g,r = cv2.split(resized_img)

            # apply CLAHE
            clahe = cv2.createCLAHE(clipLimit=5)
            b_clahe_enhanced = clahe.apply(b) + 30 # increase intensity of contrast
            g_clahe_enhanced = clahe.apply(g) + 30
            r_clahe_enhanced = clahe.apply(r)

            # merge enhanced channels
            clahe_enhanced_img = cv2.merge((b_clahe_enhanced, g, r_clahe_enhanced))

            grayscale = cv2.cvtColor(clahe_enhanced_img, cv2.COLOR_BGR2GRAY)

            if data_class == 'normal':
                train_data.append([grayscale, 0])
            elif data_class == 'pure_amd':
                train_data.append([grayscale, 1])
            elif data_class == 'amd_w_others':
                train_data.append([grayscale, 2])

In [ ]:
training_data('../data/preprocessing/filtered-data/ODIR/Pure AMD', 'pure_amd')
#training_data('../data/preprocessing/filtered-data/ODIR/AMD w others', 'amd_w_others')
#training_data('../data/preprocessing/filtered-data/ODIR/Normal', 'normal')

#training_data('../data/preprocessing/filtered-data/RFMiD/Pure AMD', 'pure_amd')
#training_data('../data/preprocessing/filtered-data/RFMiD/AMD w others', 'amd_w_others')
#training_data('../data/preprocessing/filtered-data/RFMiD/Normal', 'normal')

#training_data('../data/preprocessing/filtered-data/STARE/AMD w others', 'amd_w_others')
#training_data('../data/preprocessing/filtered-data/STARE/Normal', 'normal')

In [9]:
import random
random.shuffle(train_data)

In [10]:
x = []
y = []

for images, labels in train_data:
    x.append(images)
    y.append(labels)

In [11]:
# Saving Data
import pickle

pickle_out = open("../data/pickles/x.pickle","wb")
pickle.dump(x, pickle_out)
pickle_out.close()

pickle_out = open("../data/pickles/y.pickle","wb")
pickle.dump(y, pickle_out)
pickle_out.close()